# Variables

In [502]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [503]:
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/mnt/g/Ai agents course/LangGraph/venv/lib/python3.12/site-packages', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..']


# Testing

In [504]:
df_task = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [505]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\nNhận dạng chữ viết tay ...",Link
1,2,Bối cảnh của vấn đề:\nTrong thời đại thông tin...,Link
2,3,Bối cảnh của vấn đề:\nĐây là một nhiệm vụ hồi ...,Link
3,4,Bối cảnh của vấn đề:\nĐiều quan trọng là các c...,Link
4,5,Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thự...,Link


In [506]:
task = df_task["Task"][4]

In [507]:
task

'Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thực tế như tổng hợp nội dung hình ảnh, phân loại album, hay trích xuất đặc trưng nội dung cho ảnh tổ hợp, cần xác định chủ đề chính của một ảnh ghép từ nhiều ảnh con. Bài toán này đòi hỏi ứng dụng không chỉ nhận diện được nội dung của từng ảnh con, mà còn cần xác định chủ đề chiếm ưu thế trong toàn bộ ảnh tổng thể.\n\nYêu cầu cụ thể cần đạt được:\nMỗi đầu vào là một ảnh tổ hợp, ghép từ 9 ảnh màu nhỏ (3x3).\nNhiệm vụ của ứng dụng là:\nXác định chủ đề chiếm số lượng nhiều nhất trong 9 ảnh con.\nTrường hợp có nhiều chủ đề xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó làm kết quả.\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột thư mục có tên là "images" chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) (3 hàng × 3 cột).\n\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\ncó các cột:\nfile_name: tên file ảnh\nprediction: nhãn củ

In [508]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [509]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [510]:
class WebScraperParameters(BaseModel):
    url: str

In [511]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [512]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [513]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY
)

In [514]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [515]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [516]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [517]:
state = {
    "task": HumanMessage(content=task),
}

In [518]:
state = classify_agent(state)

# Modeling Agent

In [519]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [520]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [521]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [522]:
res = model_selection_agent(state)

/tmp/ipykernel_27136/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_27136/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [523]:
pprint.pprint(res.models)

[ModelInfo(model='Phân loại hình ảnh', url='https://huggingface.co/zhaospei/Model_7'),
 ModelInfo(model='Phân loại hình ảnh thành 100 lớp đối tượng chi tiết (fine-grained)', url='https://huggingface.co/zhaospei/Model_6'),
 ModelInfo(model='Nhận diện ảnh chữ viết tay', url='https://huggingface.co/thanhtlx/image_classification_01')]


In [524]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [525]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...

Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/zhaospei/Model_7'}`
responded: I understand the task. You want me to select the best-fit model for a given task based on the data retrieved from the provided tool. The task is to determine the dominant theme in a collage image composed of 9 smaller images.

I will start by retrieving the details of each candidate model using the provided tool.





🖼️ Mô hình Phân loại Ảnh - ViT Fine-tuned trên CIFAR-10
📝 Mô tả
Đây là mô hình Vision Transformer (ViT) được fine-tuned từ mô hình khác trên tập dữ liệu CIFAR-10. Mô hình được huấn luyện để phân loại ảnh vào 10 lớp khác nhau, mỗi lớp đại diện cho một danh mục đối tượng cụ thể.
📌 Nhiệm vụ
Loại bài toán: Phân loại ảnh (Image Classification)
Số lớp: 10 (Tương ứng nhãn) CIFAR-10
📥 Đầu vào
Định dạng: Ảnh màu RGB
Kích thước ảnh: 224x224 pixels
📤 Đầu ra
Định dạng: Xác suất cho mỗi lớp (logits)
Kiểu dữ liệu: Tensor có kích thước [batch_size, 10]
Ý

In [526]:
pprint.pprint(res2['output'])

('To solve this task, we need to analyze the details of each model and select '
 'the best one that fits the requirements.\n'
 '\n'
 'The task is to determine the dominant theme in a collage image composed of 9 '
 'smaller images. The input is a 288x288 image, which is a 3x3 grid of 32x32 '
 'images. The output should be a single label representing the most dominant '
 'theme among the 9 sub-images.\n'
 '\n'
 "Let's analyze the models:\n"
 '\n'
 '1. **Phân loại hình ảnh** — https://huggingface.co/zhaospei/Model_7\n'
 '   - Description: This is a ViT model fine-tuned on CIFAR-10 dataset for '
 'image classification into 10 classes.\n'
 '   - Input format: RGB images of size 224x224.\n'
 '   - Output format: Probabilities for each class.\n'
 '   - Library requirements: transformers, torch, torchvision.\n'
 '   - Example code: Provided.\n'
 '\n'
 '2. **Phân loại hình ảnh thành100 lớp đối tượng chi tiết (fine-grained)** — '
 'https://huggingface.co/zhaospei/Model_6\n'
 '   - Description: T

## Output Parser Agent

In [527]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

In [528]:
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

In [529]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [530]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [531]:
ans = output_parser_agent(state)

In [532]:
pprint.pprint(res2['output'])

('To solve this task, we need to analyze the details of each model and select '
 'the best one that fits the requirements.\n'
 '\n'
 'The task is to determine the dominant theme in a collage image composed of 9 '
 'smaller images. The input is a 288x288 image, which is a 3x3 grid of 32x32 '
 'images. The output should be a single label representing the most dominant '
 'theme among the 9 sub-images.\n'
 '\n'
 "Let's analyze the models:\n"
 '\n'
 '1. **Phân loại hình ảnh** — https://huggingface.co/zhaospei/Model_7\n'
 '   - Description: This is a ViT model fine-tuned on CIFAR-10 dataset for '
 'image classification into 10 classes.\n'
 '   - Input format: RGB images of size 224x224.\n'
 '   - Output format: Probabilities for each class.\n'
 '   - Library requirements: transformers, torch, torchvision.\n'
 '   - Example code: Provided.\n'
 '\n'
 '2. **Phân loại hình ảnh thành100 lớp đối tượng chi tiết (fine-grained)** — '
 'https://huggingface.co/zhaospei/Model_6\n'
 '   - Description: T

In [533]:
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_sample_code)

('This is a ViT model fine-tuned on CIFAR-100 dataset for image classification '
 'into 100 classes. The model is expected to classify images into one of 100 '
 'different themes, which could potentially align with the themes present in a '
 '3x3 grid of images.')
('RGB images of size 224x224. However, the task requires a 288x288 image '
 'composed of 9 smaller 32x32 images. The input image would need to be '
 "preprocessed to fit the model's expected input size.")
('Probabilities for each class. The output is a tensor of size [batch_size, '
 '100] representing the logits for each class. The predicted label is '
 'determined by taking the argmax of the logits.')
('import torch\n'
 'from transformers import ViTForImageClassification, ViTImageProcessor\n'
 'from PIL import Image\n'
 '\n'
 '# Load image\n'
 'image = Image.open("path_to_your_image.jpg")\n'
 '\n'
 '# Load processor and model from Hugging Face\n'
 'processor = ViTImageProcessor.from_pretrained("zhaospei/Model_6")\n'
 'model 